 # Plotly Animation video

In [6]:
from moviepy.editor import ImageSequenceClip

ModuleNotFoundError: No module named 'moviepy.editor'

In [5]:

# Import libraries
import pandas as pd
#import wbdata as wbdata
#import plotly.express as px
from moviepy.editor import ImageSequenceClip
import numpy as np

ModuleNotFoundError: No module named 'moviepy.editor'

In [2]:
# Import data
# Define the indicators for population(SP.POP.TOTL) and GDP per capita(NY.GDP.PCAP.CD)
indicator = {'SP.POP.TOTL': 'Population, total', 'NY.GDP.PCAP.CD': 'GDP per capita (current US$)'}

# Fetch data for all countries
data= wbdata.get_dataframe(indicator)
data.reset_index(inplace=True)
data.rename(columns={'country': 'Country', 'date': 'Year'}, inplace=True)
data['Year'] = pd.to_numeric(data['Year'])

# Handle missing values
data['gdp_per_capita'] = data['gdp_per_capita'].fillna(0)
data['population'] = data['population'].fillna(0)

# Filter data for valid years
df = data[(data['Year'] <=2023) & (data['Year'] >=1960)]
df.sort_values(by='Year', inplace=True)

# Ensure no missing values in 'gdp_per_capita and filter years for 2000 onwards
df_cleaned = df[(df['gdp_per_capita'].notna()) & (df['Year'] >=1960)].copy()

# Categorize 'gdp_per_capita' into bins
bins = [0, 1000, 5000,10000, 20000, 50000, np.inf]
labels = ['0-1000', '1000-5000', '5000-10000', '10000-20000', '20000-50000', '50000+']
df_cleaned['gdp_per_capita_bins'] = pd.cut(df_cleaned['gdp_per_capita'],
                                           bins=bins, labels=labels,right=False
)      
# Define the category order and consistant color mapping
category_orders= {'gdp_category': labels}

# chose a qualitative color palette with distinct colors
colors = px.colors.qualitative.Set1[:len(labels)]
# If not enough colors, extend with another palette
if len(colors) < len(labels):
                  colors+= px.colors.qualitative.Set2[:len(labels)- len(colors)] 
                  
# Map each category to a specific color 
color_dicrete_map = dict(zip(labels, colors))

# Directory to save frames
frames_dir = 'choropleth_frames_map'
# Remove if already exists
if os.path.exists(frames_dir):
                  import shutil
                  shutil.rmtree(frames_dir)
os.makedirs(frames_dir, exist_ok=True)

# Save each frame as an image file
unique_years = sorted(df_cleaned['Year'].unique())
for i, year in enumerate(unique_years):
                  # filter data for the current year
                  filtered_df = df_cleaned[df_cleaned['Year']== year]
                  
# Create a DataFrame that contains all categories for the current year
all_categories_df = pd.DataFrame([
                  'Country': [None]*len(labels),
                  'Year': [year]*len(labels),
                  'gdp_per_capita': [None]*len(labels),
                  'gdp_category': labels,
])        

# combine filtered_df with all_categories_df
filtered_df_with_all_categories = pd.concat([filtered_df, all_categories_df], ignore_index=True)

# Create the choropleth map for the current year using 'gdp_category'
frame_fig = px.choropleth(filtered_df_with_all_categories,
                          locations='Country',
                          color = 'gdp_category',
                          hover_date={'gdp_per_capita': True},
                          locationmode='country names',
                          color_discrete_map=color_discrete_map,
                          category_orders=category_orders,
                          labels={
                                            'gdp_category':'GDP per Capita(USD)',
                                            'gdp_per_capita': 'GDP per Capita',
                          },
)
# frame_fig.update_layout(margine={'r':0, 't':0,'l':0,'b':0})

# Add country borders and adjust the map view
frame_fig.update_goes(
                  showcountries=True,
                  countrycolor='gray',
                  projection_type='natural earth',
)
# Adjust figure layout
frame_fig.update_layout(
                  # height=600, 
                  title_text=f'GDP Per Capita by Countries - {int(year)}',
                  title_x=0.1, # center the title text
                  title_y=0.95, # Adjust the tilte position
)
# Size of the map
frame_fig.update_layout(
                  width=1920, height=1080
)
# Increase the font size of the title and legend
frame_fig.update_layout(
                  title_font.size=50,
                  legend_title_font_size=34,
                  legend_font_size=22,
)
# Add text annotation for the source
frame_fig.add_annotation(
                  text='''Source : World Bank Data\n
                  Auther: Abid Hussain\n'''
                  xref='paper', yref='paper', 
                  x=0.99, y=0.05,
                  showarrow=false,
                  font=dict(size=20, color='gray'),              
)
# Save the frame image using kaleido
frame_filename= os.path.join(frames_dir, f'frame_{i:03d}.png')
frame_fig.write_image(frame_filename, engine='kaleido', scale=2)

# Collect image file paths
image_files = [
     os.path.join(frames_dir, f)
     for f in sorted(os.listdir(frames_dir))
     if f.endswith('.png')            
]

fps= 2.0  #frame per second

# Create the clip from image sequence 
clip = ImageSequenceClip(image_files, fps=fps)

# Directory to save the video
output_dir = 'D:/Data_Science data/Python_ka_Chila/01_plots/choropleth_map_animation'
os.makedirs(output_dir, exist_ok=True)

# optional: Save HD video with heigher bitrate
clip.write_videofile(
                  os.path.join(output_dir, 'animated_choropleth_map_HD.mp4'),
                  fps=fps,
                  codec='libx264',
                  preset='medium',
                  bitrate='3000K',
                  audio=False,
                  ffmpeg_params=['-pix_fmt','yuv420p']
)
fps=1.0 # Frames per second

# Create the clip from image sequence
clip = ImageSequenceClip(image_files, fps=fps)
# Save as GIF
clip.write_gif(
          os.path.join(output_dir, 'animated_choropleth_map.gif')
          fps=fps
)
# Display the GIf file from the output directory ( if running in a notebook)
from IPython.display import Image as IPythonImage
IPythonImage(filename=os.path.join(output_dir, 'animated_choropleth_map.gif'))


SyntaxError: invalid syntax (3326812669.py, line 56)

In [3]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import wbdata
import plotly.express as px
from moviepy.editor import ImageSequenceClip
import shutil

# Define the indicators for population(SP.POP.TOTL) and GDP per capita(NY.GDP.PCAP.CD)
indicator = {'SP.POP.TOTL': 'Population, total', 'NY.GDP.PCAP.CD': 'GDP per capita (current US$)'}

# Fetch data for all countries
data = wbdata.get_dataframe(indicator)
data.reset_index(inplace=True)
data.rename(columns={'country': 'Country', 'date': 'Year'}, inplace=True)
data['Year'] = pd.to_numeric(data['Year'])

# Handle missing values
data['gdp_per_capita'] = data['gdp_per_capita'].fillna(0)
data['population'] = data['population'].fillna(0)

# Filter data for valid years
df = data[(data['Year'] <= 2023) & (data['Year'] >= 1960)]
df.sort_values(by='Year', inplace=True)

# Ensure no missing values in 'gdp_per_capita' and filter years for 2000 onwards
df_cleaned = df[(df['gdp_per_capita'].notna()) & (df['Year'] >= 1960)].copy()

# Categorize 'gdp_per_capita' into bins
bins = [0, 1000, 5000, 10000, 20000, 50000, np.inf]
labels = ['0-1000', '1000-5000', '5000-10000', '10000-20000', '20000-50000', '50000+']
df_cleaned['gdp_per_capita_bins'] = pd.cut(df_cleaned['gdp_per_capita'],
                                           bins=bins, labels=labels, right=False)

# Define the category order and consistent color mapping
category_orders = {'gdp_per_capita_bins': labels}

# Choose a qualitative color palette with distinct colors
colors = px.colors.qualitative.Set1[:len(labels)]
# If not enough colors, extend with another palette
if len(colors) < len(labels):
    colors += px.colors.qualitative.Set2[:len(labels) - len(colors)]

# Map each category to a specific color 
color_discrete_map = dict(zip(labels, colors))

# Directory to save frames
frames_dir = 'choropleth_frames_map'
# Remove if already exists
if os.path.exists(frames_dir):
    shutil.rmtree(frames_dir)
os.makedirs(frames_dir, exist_ok=True)

# Save each frame as an image file
unique_years = sorted(df_cleaned['Year'].unique())
for i, year in enumerate(unique_years):
    # Filter data for the current year
    filtered_df = df_cleaned[df_cleaned['Year'] == year]
    
    # Create a DataFrame that contains all categories for the current year
    all_categories_df = pd.DataFrame({
        'Country': [None] * len(labels),
        'Year': [year] * len(labels),
        'gdp_per_capita': [None] * len(labels),
        'gdp_per_capita_bins': labels
    })
    
    # Combine filtered_df with all_categories_df
    filtered_df_with_all_categories = pd.concat([filtered_df, all_categories_df], ignore_index=True)

    # Create the choropleth map for the current year using 'gdp_per_capita_bins'
    frame_fig = px.choropleth(filtered_df_with_all_categories,
                               locations='Country',
                               color='gdp_per_capita_bins',
                               hover_data={'gdp_per_capita': True},
                               locationmode='country names',
                               color_discrete_map=color_discrete_map,
                               category_orders=category_orders,
                               labels={
                                   'gdp_per_capita_bins': 'GDP per Capita (USD)',
                                   'gdp_per_capita': 'GDP per Capita',
                               })

    # Add country borders and adjust the map view
    frame_fig.update_geos(
        showcountries=True,
        countrycolor='gray',
        projection_type='natural earth'
    )

    # Adjust figure layout
    frame_fig.update_layout(
        title_text=f'GDP Per Capita by Countries - {int(year)}',
        title_x=0.1,  # Center the title text
        title_y=0.95,  # Adjust the title position
        width=1920, height=1080,
        title_font_size=50,
        legend_title_font_size=34,
        legend_font_size=22
    )

    # Add text annotation for the source
    frame_fig.add_annotation(
        text='''Source: World Bank Data\nAuthor: Abid Hussain''',
        xref='paper', yref='paper', 
        x=0.99, y=0.05,
        showarrow=False,
        font=dict(size=20, color='gray')
    )

    # Save the frame image using kaleido
    frame_filename = os.path.join(frames_dir, f'frame_{i:03d}.png')
    frame_fig.write_image(frame_filename, engine='kaleido', scale=2)

# Collect image file paths
image_files = [
    os.path.join(frames_dir, f)
    for f in sorted(os.listdir(frames_dir))
    if f.endswith('.png')
]

fps = 2.0  # Frames per second

# Create the clip from image sequence
clip = ImageSequenceClip(image_files, fps=fps)

# Directory to save the video
output_dir = 'D:/Data_Science data/Python_ka_Chila/01_plots/choropleth_map_animation'
os.makedirs(output_dir, exist_ok=True)

# Save HD video with higher bitrate
clip.write_videofile(
    os.path.join(output_dir, 'animated_choropleth_map_HD.mp4'),
    fps=fps,
    codec='libx264',
    preset='medium',
    bitrate='3000K',
    audio=False,
    ffmpeg_params=['-pix_fmt', 'yuv420p']
)

# Optional: Save as GIF
clip.write_gif(
    os.path.join(output_dir, 'animated_choropleth_map.gif'),
    fps=fps
)

# Display the GIF file (if running in a Jupyter Notebook)
from IPython.display import Image as IPythonImage
IPythonImage(filename=os.path.join(output_dir, 'animated_choropleth_map.gif'))


ModuleNotFoundError: No module named 'moviepy.editor'